# Creating Custom String Field for Salesforce Instance

Given an input company list, generate dataframe of affiliated individuals at the board/advisory level, and then collapse it down to a string for four filtering conditions:

- Current Board Members
- Former Board Members
- Current Board Advisors/Observers
- Former Board Advisors/Observers

In [1]:
# Import relevant libraries
import requests
import json
from json import JSONDecodeError
import pandas as pd
from pandas import json_normalize 
import csv
import time

# P1s Crunchbase API user key
from user_key import userkey

## All helper functions

In [2]:
def url_count(query, query_type): 
    '''
    Return the total number of results of a query, and then, deserialize the results to a Python dictionary object.
    ''' 
    
    # POST method with API URL, query_type as a parameter, and passing query as json.
    r = requests.post("https://api.crunchbase.com/api/v4/searches/" + query_type, params=userkey, json=query)
    
    # Return total number of results of query
    return json.loads(r.text)["count"]


def url_extraction(query, query_type):    
    '''
    Return the results for a query, and then, deserialize to a Python dictionary object.
    ''' 
    
    # Create global raw variable. This ensures that it can be updated if the API call needs to loop.
    global raw   
    
    # POST method with API URL, query_type as a parameter, and passing query as json.
    r = requests.post("https://api.crunchbase.com/api/v4/searches/" + query_type, params=userkey, json=query)
    
    # Return results of query
    result = json.loads(r.text)
    
    # Normalize semi-structured JSON data into a flat table, forcing it to fit into a relational data structure.
    normalized_raw = json_normalize(result['entities'])
    
    # Append normalized entity results to global raw variable
    raw = raw.append(normalized_raw, ignore_index=True)


def autocompletes(query, collection_ids_list=None, limit=None):
    '''
    Suggests matching Identifier entities based on the query and entity_def_ids provided.
    
    QUERY
    Value to perform the autocomplete search with.
    
    COLLECTION_IDS_LIST
    A comma separated list of collection ids to search against. 
    Leaving this blank means it will search across all identifiers. 
    Entity defs can be constrained to specific facets by providing them as facet collections. 
    Relationship collections will resolve to their underlying entity def.
    Collection ids are: organizations, people, funding_rounds, acquisitions, investments,
    events, press_references, funds, event_appearances, ipos, ownerships, categories, 
    category_groups, locations, jobs
    
    LIMIT
    Number of results to retrieve; default = 10, max = 25
    '''
    # Create parameter dictionary to pass into POST method
    params = {**userkey, "query": query}
    
    # Add input collection ids to parameters dictionary
    if collection_ids_list and type(collection_ids_list) == list:
        params.update({"collection_ids": collection_ids_list})
    
    # Add input limit to parameters dictionary
    if limit and type(limit) == int:
        params.update({"limit": limit})
        
    # POST method with API URL, query_type as a parameter, and passing query as json.
    r = requests.get("https://api.crunchbase.com/api/v4/autocompletes", params=params)
    
    # Return results of query
    result = json.loads(r.text)
    
    # Normalize semi-structured JSON data into a flat table, forcing it to fit into a relational data structure.
    normalized_result = json_normalize(result["entities"])
    
    # Return results of autocompletes query as pandas dataframe
    return pd.DataFrame.from_dict(normalized_result)

def makequery_board_affiliations(uuid_list, limit=1000):
    '''
    Job Search: Board Affiliations
    - Organization includes list of `uuid` values
    - Excludes `employee` and `executive` level jobs
    '''
    query = {
        "field_ids": [ # ADD FIELD IDS HERE
            "entity_def_id",
            "identifier",
            "job_type",
            "name",
            "organization_identifier",
            "person_identifier",
            "short_description",
            "is_current",
            "started_on",
            "ended_on",
            "title",
            "updated_at",
            "uuid"],
        "limit": limit, # INPUT LIMIT
        "query": [ # ADD FILTERING CONDITIONS HERE
            {
                "type": "predicate",
                "field_id": "organization_identifier",
                "operator_id": "includes",
                "values": uuid_list # INPUT UUID_LIST
            },
            {
                "type": "predicate",
                "field_id": "job_type",
                "operator_id": "not_includes",
                "values": ["employee", "executive"]
            }]
    }
    return query

def primary_info(person_id, field_ids_list=['primary_job_title','primary_organization','linkedin'], card_ids_list=None):
    '''
    PERSON_ID
    UUID or permalink of desired entity
    
    FIELD_IDS
    Fields to include on the resulting entity - 
    either an array of field_id strings in JSON 
    or a comma-separated list encoded as string
    
    CARD_IDS
    Cards to include on the resulting entity - 
    array of card_id strings in JSON encoded as string\ 
    Card Ids for Person: [degrees, event_appearances, fields, 
    founded_organizations, jobs, participated_funding_rounds, 
    participated_funds, participated_investments, partner_funding_rounds, 
    partner_investments, press_references, primary_job, primary_organization]
    '''
    # Create parameter dictionary to pass into POST method
    params = {**userkey}
    
    # Add input field ids to parameters dictionary
    if field_ids_list and type(field_ids_list) == list:
        string = ""
        for item in field_ids_list:
            string += item + ","
        string = string[:-1]
        params.update({"field_ids":string})
    
    # Add input cards ids to parameters dictionary
    if card_ids_list and type(card_ids_list) == list:
        string = ""
        for item in card_ids_list:
            string += item + ","
        string = string[:-1]
        params.update({"card_ids":string})
        
    # POST method with API URL, query_type as a parameter, and passing query as json.    
    r = requests.get("https://api.crunchbase.com/api/v4/entities/people/" + person_id, params=params)
    
    # Return results of query
    result = json.loads(r.text)
    
    # Pull uuid of searched individual
    uuid = result["properties"]["identifier"]["uuid"]
    
    # Pull LinkedIn URL from json results (if it exists)
    try:
        linkedin = result["properties"]["linkedin"]["value"]
    except:
        linkedin = "NA"
    
    # Pull primary job title from json results (if it exists)
    try:
        title = result["properties"]["primary_job_title"]
    except:
        title = "NA"
    
    # Pull primary organization from json results (if it exists)
    try:
        org = result["properties"]["primary_organization"]["value"]
    except:
        org = "NA"
    return {uuid:title}, {uuid:org}, {uuid:linkedin}

## Pull uuids of list of companies, using `autocompletes` helper function

In [3]:
companies = pd.read_csv("input/Boardroom Allies Tracker - company_search_terms.csv").Company.to_list()
#companies = pd.read_csv("input/CONFIDENTIAL Cloud 100 CEO Roundtable Invites - company_search_terms.csv").Search.to_list()

companies.sort()
uuid_companies = []
counter = 0
for company in companies:
    counter += 1
    found = autocompletes(company, ["organizations"], limit=1)
    uuid_companies.append(found["identifier.uuid"][0])
    
    # Anonymize print statements, just count.
    print(counter, end=" ")
    
    #print('*'*50)
    #print("{}) Searching for {}".format(counter, company.upper()))
    #print('*'*50)
    #print("Found {} !!!!!!!\nDESCRIPTION: {}".format(found["identifier.value"][0].upper(), found["short_description"][0]))
print('DONE!')

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 DONE!


## Pull job affiliations of company list, using `makequery_board_affiliations` helper function

In [4]:
# Make query of current/former board affiliations for companies
query = makequery_board_affiliations(uuid_companies)

# Global raw variable
raw = pd.DataFrame()

# Run query w/ API call, which populates dataframe with query results
url_extraction(query, "jobs")

# Filter down the query dataframe to usable fields
board_affiliations = raw[["properties.organization_identifier.value",  # Company name
                          "properties.person_identifier.uuid", # Person UUID
                          "properties.person_identifier.value",  # Person name
                          "properties.title",  # Job title of board affiliation
                          "properties.job_type",  # Crunchbase job_type
                          "properties.is_current", # Boolean of whether job is current or not
                          "properties.started_on.value", # Job start date
                          "properties.ended_on.value", # Job end date
                          "properties.updated_at"] # When Crunchbase last updated the record
                        ].sort_values(["properties.organization_identifier.value"]) # Sory by company name

# Get UUIDs of people
uuid_board_members = list(set(board_affiliations["properties.person_identifier.uuid"].to_list()))

# Display
#board_affiliations.head()
print('Total records found: {}'.format(board_affiliations.shape[0]))

Total records found: 464


## Add primary job title, primary organization, and LinkedIn to `board_affiliations` dataframe, using `primary_info` helper function

In [5]:
# Test example
My_Test_UUID = uuid_board_members[0]

# The function returns 3 dictionaries
if len(primary_info(My_Test_UUID)) == 3:
    print('SUCCESS!')

SUCCESS!


In [6]:
# Start with empty dictionnaries
all_titles = {}
all_orgs = {}
all_linkedin = {}
no_primary_info = []

# For each API call, update dictionary if it's not empty
i = 0
print('Count of API calls, number of unique individuals found in query:')
while i < len(uuid_board_members):
    print(i,end=" ")
    person = uuid_board_members[i]
    try: 
        # API Call
        primary_job_title, primary_org, linkedin = primary_info(person)
        
        # Update job title dictionary as long as its not equal to "NA"
        if primary_job_title[person] != "NA":
            all_titles.update(primary_job_title)
        
        # Update organization dictionary as long as its not equal to "NA"
        if primary_org[person] != "NA":
            all_orgs.update(primary_org)
        
        # Update LinkedIn dictionary as long as its not equal to "NA"
        if linkedin[person] != "NA":
            all_linkedin.update(linkedin)
        
        # If any are equal to "NA", store in no_primary_info list for safekeeping.
        if primary_job_title[person] == "NA" or primary_org[person] == "NA" or linkedin[person] == "NA":
            no_primary_info.append(person)
        
        # Continue looping
        i += 1
        
    except JSONDecodeError:
        print('[From Crunchbase: Usage limit exceeded. Pause for 5 seconds, and continue.]',end =" ")
        time.sleep(5)

# Add primary title, organization, and LinkedIn to `board_affiliations` dataframe.
board_affiliations['Primary_Job_Title'] = board_affiliations['properties.person_identifier.uuid'].map(all_titles)
board_affiliations['Primary_Organization'] = board_affiliations['properties.person_identifier.uuid'].map(all_orgs)
board_affiliations['LinkedIn'] = board_affiliations['properties.person_identifier.uuid'].map(all_linkedin)

# Create csv file 
board_affiliations.to_csv("output/board_affiliations.csv")

# Pull unique list of company names from series
company_list = list(set(board_affiliations["properties.organization_identifier.value"].to_list()))
company_list.sort()

# Count of how many are missing Title, Organization, or LinkedIn
print('\n\n{} out of {} records are missing either a primary job title, primary organization, or LinkedIn url.'.format(len(no_primary_info),i-1))

Count of API calls, number of unique individuals found in query:
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 2

## Current board members

In [13]:
board_members_dict = {}

# For each company
for org in company_list:
    
    # Pull dataframe of CURRENT BOARD MEMBERS
    board_members_df = board_affiliations[(board_affiliations["properties.organization_identifier.value"]==org) & 
                                          (board_affiliations["properties.is_current"]) & 
                                          (board_affiliations["properties.job_type"]=="board_member")
                                         ].sort_values(["properties.person_identifier.value"])
    
    # Collapse individual names to list
    names = board_members_df["properties.person_identifier.value"].to_list()
    
    # Collapse individual organizations to list
    companies = board_members_df["Primary_Organization"].to_list()
    
    # Start with empty string
    board_string = ""
    
    # Exclude if there are no individuals affiliated
    if names != []:
        
        # Make temp dictionary of name:org
        board_info = dict(zip(names, companies))
        
        # Add them to string
        for name, company in sorted(board_info.items()):
            
            # If individual does not have a primary organization
            if pd.isna(company):
                board_string += name + "; "
            
            # If individual has a primary organziation, place into parentheses
            else:
                board_string += name + " (" + company + "); "
        
        # Remove trailing semicolon and remove extra commas
        board_string = board_string[:-2].replace(',', '')
    
    # Add string to main dictionary 
    board_members_dict[org] = board_string

board_members_dict_1 = board_members_dict

## Former board members

In [14]:
board_members_dict = {}

# For each company
for org in company_list:
    
    # Pull dataframe of FORMER BOARD MEMBERS
    board_members_df = board_affiliations[(board_affiliations["properties.organization_identifier.value"]==org) &
                                          (board_affiliations["properties.is_current"]==False) &
                                          (board_affiliations["properties.job_type"]=="board_member")
                                         ].sort_values(["properties.person_identifier.value"])
    # Collapse individual names to list
    names = board_members_df["properties.person_identifier.value"].to_list()
    
    # Collapse individual organizations to list
    companies = board_members_df["Primary_Organization"].to_list()
    
    # Start with empty string
    board_string = ""
    
    # Exclude if there are no individuals affiliated
    if names != []:
        
        # Make temp dictionary of name:org
        board_info = dict(zip(names, companies))
        
        # Add them to string
        for name, company in sorted(board_info.items()):
            
            # If individual does not have a primary organization
            if pd.isna(company):
                board_string += name + "; "
            
            # If individual has a primary organziation, place into parentheses
            else:
                board_string += name + " (" + company + "); "
        
        # Remove trailing semicolon and remove extra commas
        board_string = board_string[:-2].replace(',', '')
    
    # Add string to main dictionary 
    board_members_dict[org] = board_string

board_members_dict_2 = board_members_dict

## Current board advisors or board observers

In [15]:
board_members_dict = {}

# For each company
for org in company_list:
    
    # Pull dataframe of CURRENT BOARD ADVISORS/OBSERVERS
    board_members_df = board_affiliations[(board_affiliations["properties.organization_identifier.value"]==org) &
                                          (board_affiliations["properties.is_current"]) &
                                          (board_affiliations["properties.job_type"]!="board_member")
                                         ].sort_values(["properties.person_identifier.value"])
    
    # Collapse individual names to list
    names = board_members_df["properties.person_identifier.value"].to_list()
    
    # Collapse individual organizations to list
    companies = board_members_df["Primary_Organization"].to_list()
    
    # Start with empty string
    board_string = ""
    
    # Exclude if there are no individuals affiliated
    if names != []:
        
        # Make temp dictionary of name:org
        board_info = dict(zip(names, companies))
        
        # Add them to string
        for name, company in sorted(board_info.items()):
            
            # If individual does not have a primary organization
            if pd.isna(company):
                board_string += name + "; "
            
            # If individual has a primary organziation, place into parentheses
            else:
                board_string += name + " (" + company + "); "
        
        # Remove trailing semicolon and remove extra commas
        board_string = board_string[:-2].replace(',', '')
    
    # Add string to main dictionary 
    board_members_dict[org] = board_string

board_members_dict_3 = board_members_dict

## Former board advisors or board observers

In [16]:
board_members_dict = {}
# For each company
for org in company_list:
    
    # Pull dataframe of FORMER BOARD ADVISORS/OBSERVERS
    board_members_df = board_affiliations[(board_affiliations["properties.organization_identifier.value"]==org) &
                                          (board_affiliations["properties.is_current"] == False) &
                                          (board_affiliations["properties.job_type"]!="board_member")
                                         ].sort_values(["properties.person_identifier.value"])
    
    # Collapse individual names to list
    names = board_members_df["properties.person_identifier.value"].to_list()

    # Collapse individual organizations to list
    companies = board_members_df["Primary_Organization"].to_list()

    # Start with empty string
    board_string = ""
    
    # Exclude if there are no individuals affiliated
    if names != []:
        
        # Make temp dictionary of name:org
        board_info = dict(zip(names, companies))
        
        # Add them to string
        for name, company in sorted(board_info.items()):
            
            # If individual does not have a primary organization
            if pd.isna(company):
                board_string += name + "; "
            
            # If individual has a primary organziation, place into parentheses
            else:
                board_string += name + " (" + company + "); "
        
        # Remove trailing semicolon and remove extra commas
        board_string = board_string[:-2].replace(',', '')
    
    # Add string to main dictionary 
    board_members_dict[org] = board_string
    
board_members_dict_4 = board_members_dict

## To CSV

In [17]:
with open('output/str_board_affiliations.csv', 'w') as f:
    for i,key in enumerate(board_members_dict.keys()):
        if i == 0:
            # Add header
            f.write("Company, Current Board Members, Former Board Members, Current Board Advisors/Observers, Former Board Advisors/Observers\n")
            f.write("%s, %s, %s, %s, %s\n" % (key, 
                                          board_members_dict_1[key], 
                                          board_members_dict_2[key], 
                                          board_members_dict_3[key], 
                                          board_members_dict_4[key]))
            continue
        f.write("%s, %s, %s, %s, %s\n" % (key, 
                                          board_members_dict_1[key], 
                                          board_members_dict_2[key], 
                                          board_members_dict_3[key], 
                                          board_members_dict_4[key]))

## Check out results

In [18]:
df = pd.read_csv('output/str_board_affiliations.csv')
df.head()

,Company,Current Board Members,Former Board Members,Current Board Advisors/Observers,Former Board Advisors/Observers
0,Atlassian,Enrique Salem (Bain Capital Ventures); Heathe...,Doug Burgum; Jay Parikh (Facebook); Kirk Bowm...,,
1,Auth0,Anders Ranum (Sapphire Ventures); David Cowan...,Sunil Nagaraj (Ubiquity Ventures),George Bischof (Meritech Capital Partners); G...,Clay Whitehead; Glenn Block (Auth0); Steve Co...
2,Canva,Keval Desai (InterWest Partners); Trevor Fols...,,Ash Fontana (Zetta Venture Partners); Bill Ta...,Casey Winters (Eventbrite)
3,Code42,Kirk Bowman (Accel); Matthew Dornquast (Code4...,,,
4,Coinbase,Barry Schuler (DFJ Growth); Brian Armstrong (...,David Marcus (Facebook); Victor Pascucci,Gavin Andresen (Bitcoin Foundation); Juan Zar...,Gopi Rangan (Sure Ventures)
